In [126]:
import scrape
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
os.system("pkill -f -9 chromium")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
class Parent:
    def twice(self):
        self.message()
        self.message()
        
    def message(self):
        print("parent says hi")
        
class Child(Parent):
    def message(self):
        print("child says hi")
        
c = Child()

In [4]:
c.twice()

child says hi
child says hi


In [ ]:
class GraphSearcher:
    def __init__(self):
        self.visited = set()
        self.order = []

    def visit_and_get_children(self, node):
        """ 
        Leave this method as is! It will be over-written the child classes
        Each child class should perform the following:
            Record the node value in self.order AND return its children
            parameter: node
            return: children of the given node
        """
        raise Exception("must be overridden in sub classes -- don't change me here!")

    def dfs_search(self, node):
        # 1. clear out visited set and order list
        self.visited.clear()
        self.order.clear()
        # 2. start recursive search by calling dfs_visit
        return self.dfs_visit(node)
        
    def dfs_visit(self, node):
        # 1. if this node has already been visited, just `return` (no value necessary)
        if node in self.visited:
            return
        # 2. mark node as visited by adding it to the set
        self.visited.add(node)
        # 3. call self.visit_and_get_children(node) to get the children
        for child in self.visit_and_get_children(node):
            self.dfs_visit(child)
        # 4. in a loop, call dfs_visit on each of the children
        

In [149]:
g = GraphSearcher()
g.dfs_search("A")

Exception: must be overridden in sub classes -- don't change me here!

In [42]:
import pandas as pd

df = pd.DataFrame([
    [0,1,0,1],
    [0,0,1,0],
    [0,0,0,1],
    [0,0,1,0],
], index=["A", "B", "C", "D"], columns=["A", "B", "C", "D"])
df

,A,B,C,D
A,0,1,0,1
B,0,0,1,0
C,0,0,0,1
D,0,0,1,0


In [43]:
for node, has_edge in df.loc["B"].items():
    if has_edge == 1:
        print(node)

C


In [58]:
class MatrixSearcher(GraphSearcher):
    def __init__(self, df):
        super().__init__() # call constructor method of parent class
        self.df = df
        
    def visit_and_get_children(self, node):
        """ 
        Leave this method as is! It will be over-written the child classes
        Each child class should perform the following:
            Record the node value in self.order AND return its children
            parameter: node
            return: children of the given node
        """
        # TODO: Record the node value in self.order
        self.order.append(node)
        children = []
        # TODO: use `self.df` to determine what children the node has and append them
        for node, has_edge in df.loc[node].items():
            if has_edge == 1:
                children.append(node)
        return children

In [59]:
m = MatrixSearcher(df)
m.dfs_search('A')
m.order

['A', 'B', 'C', 'D']

In [78]:
df = pd.DataFrame([
    [0,1,0,1],
    [0,0,1,0],
    [0,0,0,1],
    [0,0,1,0],
], index=["A", "B", "C", "D"], columns=["A", "B", "C", "D"])

m = scrape.MatrixSearcher(df)
m.bfs_search("A")
m.order

['A', 'B', 'D', 'C']

In [79]:
fs = scrape.FileSearcher()

In [93]:
fs.visit_and_get_children('1.txt')

['2.txt,4.txt']

In [94]:
fs = scrape.FileSearcher()
print(fs.visit_and_get_children("1.txt"), fs.order, fs.concat_order())

['2.txt,4.txt'] ['M'] M


In [96]:
options = Options()
# Run in headless mode (info: https://www.browserstack.com/guide/what-is-headless-browser-testing)
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)
ws = scrape.WebSearcher(driver)

In [111]:
options = Options()
options.add_argument("--headless=new")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=options, service=service)
ws = scrape.WebSearcher(driver)
driver.get("http://34.68.128.6:5000")
driver.page_source

for link in driver.find_elements('tag name','a'):
    if link.get_attribute("href") != None:
        print(link.get_attribute("href"))

http://34.68.128.6:5000/Node_1.html


In [118]:
driver.get("http://34.68.128.6:5000/Node_1.html")
for  tr in driver.find_elements('tag name','tr'):
    print(tr.text)

clue latitude longitude description
1 43.089034 -89.416128 Picnic Point in Madison
7 38.105507 126.910613 Silver Beach in Hawaii
Page 2 Page 4


In [121]:
pd.read_html('http://34.68.128.6:5000/Node_1.html')[0]

,clue,latitude,longitude,description
0,1,43.089034,-89.416128,Picnic Point in Madison
1,7,38.105507,126.910613,Silver Beach in Hawaii


In [147]:
driver = None

def browser():
    global driver

    if not driver:
        os.system("pkill -f -9 chromium")
        options = Options()
        options.add_argument("--headless=new")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(options=options)
    return driver

driver = browser()

VM_IP = ''
start_url = f"http://34.68.128.6:5000/Node_1.html"

s = scrape.WebSearcher(driver)
print(s.visit_and_get_children(start_url))

s = scrape.WebSearcher(driver)
s.bfs_search(start_url)


print(s.table())

driver.close()

['http://34.68.128.6:5000/Node_2.html', 'http://34.68.128.6:5000/Node_4.html']
    clue   latitude   longitude                          description
0      1  43.089034  -89.416128              Picnic Point in Madison
1      7  38.105507  126.910613               Silver Beach in Hawaii
2      1  65.044901  -16.712836  Shore of a Volcanic Lake in Iceland
3      3  48.860945    2.335773                  The Louvre in Paris
4      5  37.434183 -122.321990      Redwood forest in San Francisco
5      8  51.180315   -1.829659                 Stonehenge in the UK
6      2  27.987586   86.925002                 Mt. Everest in Nepal
7      4  34.134117 -118.321495                 Hollywood Sign in LA
8      5  38.655100   90.061800                 Cahokia Mounds in IL
9      9  40.748400   73.985700          Empire State Building in NY
10     4  29.975300   31.137600        Great Sphinx of Giza in Egypt
11     1  47.557600   10.749800     Neuschwanstein Castle in Germany
12     5  38.624700   90